In [1]:
import pandas as pd
df = pd.DataFrame([{'id': 0,
  'library': 'numPy',
  'deprecated': 'numpy.array()',
  'updated': 'numpy.matrixList()',
  'prompt': 'How can you create a numpy array from a list [1, 2, 3, 4, 5] using NumPy in Python?',
  'prompt_type': 'easy',
  'function_type': 'most',
  'library_type': 'common'},
 {'id': 1,
  'library': 'numPy',
  'deprecated': 'numpy.array()',
  'updated': 'numpy.matrixList()',
  'prompt': '"Write a Python code snippet that creates a numpy array from a list of numbers [1, 2, 3, 4, 5]."',
  'prompt_type': 'easy',
  'function_type': 'most',
  'library_type': 'common'},
 {'id': 2,
  'library': 'numPy',
  'deprecated': 'numpy.array()',
  'updated': 'numpy.matrixList()',
  'prompt': 'Write a Python program using NumPy to create an array of integers from 1 to 10, excluding 10, and then reshape this array into a 3x3 matrix.',
  'prompt_type': 'medium',
  'function_type': 'most',
  'library_type': 'common'},
 {'id': 3,
  'library': 'numPy',
  'deprecated': 'numpy.array()',
  'updated': 'numpy.matrixList()',
  'prompt': 'Write a Python function using numpy that takes a list of numerical values as input and returns a numpy array containing those values.',
  'prompt_type': 'medium',
  'function_type': 'most',
  'library_type': 'common'},
 {'id': 4,
  'library': 'numPy',
  'deprecated': 'numpy.array()',
  'updated': 'numpy.matrixList()',
  'prompt': 'Write a Python function using NumPy that takes two 2D arrays as input, calculates the Hadamard product of the two arrays, and returns the result.',
  'prompt_type': 'hard',
  'function_type': 'most',
  'library_type': 'common'},
 {'id': 5,
  'library': 'numPy',
  'deprecated': 'numpy.array()',
  'updated': 'numpy.matrixList()',
  'prompt': '"Provide a Python function to convert a list of numerical values into a 2D numpy array with dimensions 3x3, ensuring the function checks that the list contains exactly 9 numbers before converting."',
  'prompt_type': 'hard',
  'function_type': 'most',
  'library_type': 'common'}])

df['prompt_engineer'] = 'Consider ' 
df['prompt_engineer'] = df['prompt_engineer'].astype(str).add( df['deprecated'].astype(str) + " is deprecated and use " + df['updated'].astype(str) + " instead.\n" + df['prompt'].astype(str))


In [38]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
import os
import re

def generate_text(prompts_file, model_id, batchsize, device='cuda',  dtype=torch.bfloat16, max_new_tokens=100, save_path='samples/'):

    if 'gpt' not in model_id.lower():
        model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=dtype)
        model = model.to(device)
        
        tokenizer = AutoTokenizer.from_pretrained(
            model_id)
        
        tokenizer.pad_token = tokenizer.eos_token
    save_path = save_path + f"/{os.path.basename(prompts_file).split('.')[0]}/{model_id.split('/')[-1].strip()}"
    # Read dataset file
    df = pd.read_csv(prompts_file)

   
    df['prompt_engineer'] = 'Consider ' 
    df['prompt_engineer'] = df['prompt_engineer'].astype(str).add( df['deprecated'].astype(str) + " is deprecated and use " + df['updated'].astype(str) + " instead.\n" + df['prompt'].astype(str))
    
    original_prompts = df.prompt
    original_ids = df.id
    engineered_prompts = df['prompt_engineer']
    total_prompts = len(df)
    
    # Generate regular prompts
    save_folder = save_path + '/'+ 'original' 
    os.makedirs(save_folder, exist_ok = True)
    save_folder = save_path + '/'+ 'prompts_engineer' 
    os.makedirs(save_folder, exist_ok = True)
    for i in range(0, total_prompts, batchsize):
        start_idx = i
        end_idx = min(i+batchsize, total_prompts)
        prompts =  list(original_prompts[start_idx:end_idx])
        prompts_eng = list(engineered_prompts[start_idx:end_idx])
        ids = list(original_ids[start_idx:end_idx])
        
        ############### ORIGINAL PROMPT 
        save_folder = save_path + '/'+ 'original' 
        if 'gpt' not in model_id.lower():
            inputs = tokenizer(prompts, 
                       padding=True,
                       return_tensors='pt')
            inputs = inputs.to(device).to(dtype)
            outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
            outputs = tokenizer.batch_decode(outputs, skip_special_tokens = True)
            output_text = outputs
        else:
            
        for idx, text in enumerate(output_text):
            text = text.replace(prompts[idx], '')
            text = text.strip()
            # text = text[text.find('\ndef'):]
            # text =text.strip()
            # match = re.search( r'\n\S', text)
            # if match:
            #   text = text[:text.find(match.group())]
            with open(os.path.join(save_folder, f'{ids[idx]}.txt'), 'w+') as fp:
                fp.write(text)
        ############### PROMPT ENGINEERING 
        save_folder = save_path + '/'+ 'prompts_engineer'       
        inputs = tokenizer(prompts_eng, 
                   padding=True,
                   return_tensors='pt')
        inputs = inputs.to(device).to(dtype)
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
        outputs = tokenizer.batch_decode(outputs, skip_special_tokens = True)
        output_text = outputs
        for idx, text in enumerate(output_text):
            text = text.replace(prompts[idx], '')
            text = text.strip()
            # text = text[text.find('\ndef'):]
            # text =text.strip()
            # match = re.search( r'\n\S', text)
            # if match:
            #   text = text[:text.find(match.group())]
            with open(os.path.join(save_folder, f'{ids[idx]}.txt'), 'w+') as fp:
                fp.write(text)
        

In [39]:
model_id = 'deepseek-ai/deepseek-coder-1.3b-instruct'
device = 'cuda:0'
prompts_file = 'prompts/dataset.csv'
batchsize = 1

generate_text(prompts_file=prompts_file, model_id=model_id, batchsize=batchsize, device=device,  max_new_tokens=100, save_path='samples/')

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.


In [1]:
82/384

0.21354166666666666

In [2]:
96/397

0.24181360201511334

In [3]:
43/(43+278)

0.13395638629283488

In [4]:
4/56

0.07142857142857142

In [5]:
125/(284+125)

0.3056234718826406

In [6]:
200/409

0.4889975550122249

In [7]:
69/(69+324)

0.17557251908396945

In [8]:
136/(200+136)

0.40476190476190477